In [1]:
import anthropic
claude_key = "claude_key"

In [17]:
from sklearn.model_selection import KFold
from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets
import datasets
import pandas as pd
import os
import logging
import nltk
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from random import sample


validate = datasets.load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/data/validate")
validate

Dataset({
    features: ['row_ids', 'summary', 'query', 'table', 'example_id'],
    num_rows: 200
})

In [18]:
import pandas as pd

def to_pandas(item):
  return pd.DataFrame(item['table']["rows"],columns=item['table']["header"])

In [19]:
# Set up the client
client = anthropic.Client(api_key = claude_key)
def create_prompt(item, question, title):
    table_str = to_pandas(item).to_markdown()
    prompt = f"The title of the table is {title}. \n\nTable data: \n\n{table_str}. \n\nQuery: \n\n{question}\n\n" 
    return prompt

In [38]:
example = validate[121]
question = example['query']
title = example["table"]['title']
print(create_prompt(example, question, title))

The title of the table is 2004 United States presidential election in Vermont - Results. 

Table data: 

|    | Presidential candidate   | Party             | Popular vote   | Percentage   |   Electoral vote |
|---:|:-------------------------|:------------------|:---------------|:-------------|-----------------:|
|  0 | John Kerry               | Democratic        | 184,067        | 58.94%       |                3 |
|  1 | George W. Bush           | Republican        | 121,180        | 38.80%       |                0 |
|  2 | Ralph Nader              | Independent       | 4,494          | 1.44%        |                0 |
|  3 | Michael Badnarik         | Libertarian       | 1,102          | 0.35%        |                0 |
|  4 | Write ins                |                   | 957            | 0.31%        |                0 |
|  5 | John Parker              | Liberty Union     | 265            | 0.09%        |                0 |
|  6 | Róger Calero             | Socialist Workers | 2

In [42]:
from anthropic import HUMAN_PROMPT, AI_PROMPT
from IPython.display import Markdown, display

summaries = []
system_prompt = "You are an AI assistant tasked with summarizing tabular data based on the user query. Provide a concise paragraph highlighting the main trends and observations from the table relevant to the query."

def generate_summary(item):
    question = item['query']
    title = item['table']["title"]
    prompt = create_prompt(item, question, title)

    # Call the completions API
    message = client.messages.create(
            model="claude-3-opus-20240229",
            max_tokens=100,
            system=system_prompt,
            temperature=0.3,
            messages=[
                {"role": "user", "content": f"{prompt}"}
            ]
    )

    # Return the summary
    return message.content[0].text

In [51]:
import time
count = 1
for item in validate:
    print(count)
    count += 1
    summary = generate_summary(item)
    summaries.append(summary)
    time.sleep(30)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

In [53]:
len(summaries)

180

In [56]:
summaries[179]

"Based on the data provided, Iowa had the highest points per game (PPG) average at 27.2 during the 1958 Big Ten Conference football season. Iowa's strong offensive performance directly correlated with their overall success, as they finished the season with an 8-1-1 overall record and a 5-1 conference record, securing the #1 rank in the conference. Additionally, Iowa ended the season ranked #2 in the AP poll,"

In [58]:
validate = validate.select(range(180))
validate = validate.add_column("generated_summary", summaries)
validate.save_to_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/claude-3")

Saving the dataset (1/1 shards): 100%|██████████| 180/180 [00:00<00:00, 2845.50 examples/s]


In [59]:
import evaluate
rouge = evaluate.load('rouge')

pred = validate['generated_summary']
ref = validate['summary']
sc = rouge.compute(predictions=pred, references=ref)

In [60]:
sc

{'rouge1': 0.4975975392508061,
 'rouge2': 0.2561522430887462,
 'rougeL': 0.38134067199503013,
 'rougeLsum': 0.3857505740567201}

In [ ]:
##### Evaluation

In [1]:
import datasets
validate = datasets.load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/claude-3")

/blue/cai6307/y.khan/conda/envs/hf-llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import evaluate
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

pred = validate['generated_summary']
ref = validate['summary']
sc = rouge.compute(predictions=pred, references=ref)
bert_score = bertscore.compute(predictions=pred, references=ref, lang = "en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import numpy as np
sc['rougeLsum'], np.mean(bert_score['f1'])

(0.3854464015869393, 0.9022718866666158)

In [6]:
import datasets
validate = datasets.load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/claude-3")

In [8]:
example = validate.filter(lambda x: x["example_id"] == "1ae05e92-4ca6-483d-9307-488d28b0b31b")
suma = example['generated_summary']
print(suma)

['Based on the given table of Utah Jazz all-time roster, there is one player who attended Brigham Young University (BYU):\n\nAndy Toolson, who wore jersey number 5, played the positions of guard and forward for the Utah Jazz in two separate stints: from 1990 to 1991 and again from 1995 to 1996. Toolson is a United States national.']
